# TensorFlow Regression Model Workflow with Script Mode and Automatic Model Tuning

Starting with TensorFlow version 1.11, you can use SageMaker's prebuilt TensorFlow containers with TensorFlow training scripts similar to those you would use outside SageMaker. This feature is named Script Mode.

In this notebook, we will use Script Mode in conjunction with TensorFlow's Eager Execution mode, which is the default execution mode of TensorFlow 2 onwards.  Eager execution is an imperative interface where operations are executed immediately, rather than building a static computational graph. Advantages of Eager Execution include a more intuitive interface with natural Python control flow and less boilerplate, easier debugging, and support for dynamic models and almost all of the available TensorFlow operations. It also features close integration with tf.keras to make rapid prototyping even easier.  

To demonstrate Script Mode, this notebook focuses on presenting a relatively complete workflow. The workflow includes local training and hosted training in SageMaker, as well as local inference and SageMaker hosted inference with a real time endpoint. Additionally, Automatic Model Tuning in SageMaker will be used to tune the model's hyperparameters.  This workflow will be applied to a straightforward regression task, predicting house prices based on the well-known Boston Housing dataset. More specifically, this public dataset contains 13 features regarding housing stock of towns in the Boston area, including features such as average number of rooms, accessibility to radial highways, adjacency to the Charles River, etc.  

To begin, we'll import some necessary packages and set up directories for training and test data.  

In [1]:
import os
import tensorflow as tf

tf.enable_eager_execution()
tf.set_random_seed(0)
tf.logging.set_verbosity(tf.logging.ERROR)

data_dir = os.path.join(os.getcwd(), 'data')
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(os.getcwd(), 'data/train')
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(os.getcwd(), 'data/test')
os.makedirs(test_dir, exist_ok=True)

# Prepare dataset

Next, we'll import the dataset. The dataset itself is small and relatively issue-free. For example, there are no missing values, a common problem for many other datasets. Accordingly, preprocessing just involves normalizing the data.    

In [2]:
from tensorflow.python.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)

x_train = (x_train - mean) / (std + 1e-8)
x_test = (x_test - mean) / (std + 1e-8)

print('x train', x_train.shape, x_train.mean(), x_train.std())
print('y train', y_train.shape, y_train.mean(), y_train.std())
print('x test', x_test.shape, x_test.mean(), x_test.std())
print('y test', y_test.shape, y_test.mean(), y_test.std())

57344/57026 [==============================] - 0s 0us/step
x train (404, 13) 2.6029783389231392e-15 0.9999999879626582
y train (404,) 22.395049504950492 9.199035423364862
x test (102, 13) 0.020826991529340172 0.9836083314719052
y test (102,) 23.07843137254902 9.123806690181466


The data is saved as Numpy files prior to both local mode training and hosted training in SageMaker.

In [3]:
import numpy as np

np.save(os.path.join(train_dir, 'x_train.npy'), x_train)
np.save(os.path.join(train_dir, 'y_train.npy'), y_train)
np.save(os.path.join(test_dir, 'x_test.npy'), x_test)
np.save(os.path.join(test_dir, 'y_test.npy'), y_test)

##  Local Mode training

Amazon SageMaker’s Local Mode training feature is a convenient way to make sure your code is working as expected before moving on to full scale, hosted training. To train in Local Mode, it is necessary to have docker-compose or nvidia-docker-compose (for GPU) installed in the notebook instance. Running following script will install docker-compose or nvidia-docker-compose and configure the notebook environment for you.

In [4]:
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/local_mode_setup.sh
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/daemon.json    
!/bin/bash ./local_mode_setup.sh

SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


Next, we'll set up a TensorFlow Estimator for Local Mode training. One of the key parameters for an Estimator is the `train_instance_type`, which is the kind of hardware on which training will run. In the case of Local Mode, we simply set this parameter to `local` to invoke Local Mode training on the CPU, or to `local_gpu` if the instance has a GPU. Other parameters of note are the algorithm’s hyperparameters, which are passed in as a dictionary, and a Boolean parameter indicating that we are using Script Mode. 

Recall that we are using Local Mode here mainly to make sure our code is working. Accordingly, instead of performing a full cycle of training with many epochs (passes over the full dataset), we'll train only for a small number of epochs to confirm the code is working properly and avoid wasting training time unnecessarily.

In [5]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

git_config = {'repo': 'https://github.com/aws-samples/amazon-sagemaker-script-mode', 
              'branch': 'master'}

model_dir = '/opt/ml/model'
train_instance_type = 'local'
hyperparameters = {'epochs': 5, 'batch_size': 128, 'learning_rate': 0.01}
local_estimator = TensorFlow(
                       git_config=git_config,
                       source_dir='tf-eager-script-mode/train_model',
                       entry_point='train.py',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-scriptmode-bostonhousing',
                       framework_version='1.13',
                       py_version='py3',
                       script_mode=True)

In [6]:
inputs = {'train': f'file://{train_dir}',
          'test': f'file://{test_dir}'}

local_estimator.fit(inputs)

Creating tmplq0rqx34_algo-1-ojron_1 ... 
Attaching to tmplq0rqx34_algo-1-ojron_12mdone
algo-1-ojron_1  | 2019-10-11 01:42:15,262 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-ojron_1  | 2019-10-11 01:42:15,268 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-ojron_1  | 2019-10-11 01:42:15,432 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-ojron_1  | 2019-10-11 01:42:15,453 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-ojron_1  | 2019-10-11 01:42:15,473 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-ojron_1  | 2019-10-11 01:42:15,488 sagemaker-containers INFO     Invoking user script
algo-1-ojron_1  | 
algo-1-ojron_1  | Training Env:
algo-1-ojron_1  | 
algo-1-ojron_1  | {
algo-1-ojron_1  |     "additional_framework_parameters": {},
algo-1-ojron_1  |     "channel_input_dirs": {
algo-1-ojron_1  

algo-1-ojron_1  | Epoch 1/5
404/404 [==============================] - 0s 772us/sample - loss: 539.0984 - val_loss: 424.8362
algo-1-ojron_1  | Epoch 2/5
404/404 [==============================] - 0s 22us/sample - loss: 356.4195 - val_loss: 256.6098
algo-1-ojron_1  | Epoch 3/5
404/404 [==============================] - 0s 21us/sample - loss: 207.4756 - val_loss: 142.9375
algo-1-ojron_1  | Epoch 4/5
404/404 [==============================] - 0s 20us/sample - loss: 119.1653 - val_loss: 92.6083
algo-1-ojron_1  | Epoch 5/5
404/404 [==============================] - 0s 20us/sample - loss: 82.4789 - val_loss: 67.8991
algo-1-ojron_1  |  - 0s - loss: 67.8991
algo-1-ojron_1  | Test MSE : 67.89910888671875
algo-1-ojron_1  | WARNING:tensorflow:Saver is deprecated, please switch to tf.train.Checkpoint or tf.keras.Model.save_weights for training checkpoints. When executing eagerly variables do not necessarily have unique names, and so the variable.name-based lookups Saver performs are error-prone.
a

## Local Mode endpoint

While Amazon SageMaker’s Local Mode training is very useful to make sure your training code is working before moving on to full scale training, it also would be useful to have a convenient way to test your model locally before incurring the time and expense of deploying it to production. One possibility is to fetch the SavedModel artifact or a model checkpoint saved in Amazon S3, and load it in your notebook for testing. We'll explore doing that in another section of this notebook below.  However, an even easier way to do this is to use the Amazon SageMaker SDK to do this work for you.

The Estimator object from the Local Mode training job can be used to deploy a model locally with a single line of code. With one exception, this code is the same as the code you would use to deploy to production. In particular, all you need to do is invoke the local Estimator's deploy method, and similarly to Local Mode training, specify the instance type as either `local_gpu` or `local` depending on whether your notebook instance is a GPU instance or CPU instance.

In [7]:
local_predictor = local_estimator.deploy(initial_instance_count=1,instance_type='local')

Attaching to tmpitj58g7f_algo-1-lxcjs_1
algo-1-lxcjs_1  | INFO:__main__:starting services
algo-1-lxcjs_1  | INFO:__main__:using default model name: model
algo-1-lxcjs_1  | INFO:__main__:tensorflow serving model config: 
algo-1-lxcjs_1  | model_config_list: {
algo-1-lxcjs_1  |   config: {
algo-1-lxcjs_1  |     name: "model",
algo-1-lxcjs_1  |     base_path: "/opt/ml/model",
algo-1-lxcjs_1  |     model_platform: "tensorflow"
algo-1-lxcjs_1  |   },
algo-1-lxcjs_1  | }
algo-1-lxcjs_1  | 
algo-1-lxcjs_1  | 
algo-1-lxcjs_1  | INFO:__main__:nginx config: 
algo-1-lxcjs_1  | load_module modules/ngx_http_js_module.so;
algo-1-lxcjs_1  | 
algo-1-lxcjs_1  | worker_processes auto;
algo-1-lxcjs_1  | daemon off;
algo-1-lxcjs_1  | pid /tmp/nginx.pid;
algo-1-lxcjs_1  | error_log  /dev/stderr info;
algo-1-lxcjs_1  | 
algo-1-lxcjs_1  | worker_rlimit_nofile 4096;
algo-1-lxcjs_1  | 
algo-1-lxcjs_1  | events {
algo-1-lxcjs_1  |   worker_connections 2048;
algo-1-lxcjs_1  | }
algo-1-lxcjs_1  | 
algo-1-lxcjs_1 

algo-1-lxcjs_1  | 2019-10-11 01:43:29.231532: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:182] Restoring SavedModel bundle.
algo-1-lxcjs_1  | 2019-10-11 01:43:29.241342: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:132] Running initialization op on SavedModel bundle.
algo-1-lxcjs_1  | 2019-10-11 01:43:29.243525: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:285] SavedModel load for tags { serve }; Status: success. Took 34161 microseconds.
algo-1-lxcjs_1  | 2019-10-11 01:43:29.243576: I tensorflow_serving/servables/tensorflow/saved_model_warmup.cc:101] No warmup data file found at /opt/ml/model/1570758139/assets.extra/tf_serving_warmup_requests
algo-1-lxcjs_1  | 2019-10-11 01:43:29.243783: I tensorflow_serving/core/loader_harness.cc:86] Successfully loaded servable version {name: model version: 1570758139}
algo-1-lxcjs_1  | 2019-10-11 01:43:29.245518: I tensorflow_serving/model_servers/server.cc:313] Running gRPC ModelServer at 0.0.0.0:

To get predictions from the local endpoint, simply invoke the Predictor's predict method.

In [8]:
local_results = local_predictor.predict(x_test[:10])['predictions']

algo-1-lxcjs_1  | 172.18.0.1 - - [11/Oct/2019:01:43:59 +0000] "POST /invocations HTTP/1.1" 200 137 "-" "-"


As a sanity check, the predictions can be compared against the actual target values.

In [9]:
local_preds_flat_list = [float('%.1f'%(item)) for sublist in local_results for item in sublist]
print('predictions: \t{}'.format(np.array(local_preds_flat_list)))
print('target values: \t{}'.format(y_test[:10].round(decimals=1)))

predictions: 	[14.3 20.5 21.4 21.1 21.7 20.1 21.8 21.6 18.7 19.5]
target values: 	[ 7.2 18.8 19.  27.  22.2 24.5 31.2 22.9 20.5 23.2]


We only trained the model for a few epochs and there is room for improvement, but the predictions so far should at least appear reasonably within the ballpark.  

To avoid having the TensorFlow Serving container running indefinitely on this notebook instance, simply gracefully shut it down by calling the `delete_endpoint` method of the Predictor object.

In [10]:
local_predictor.delete_endpoint()

Gracefully stopping... (press Ctrl+C again to force)


##  SageMaker hosted training

Now that we've confirmed our code is working locally, we can move on to use SageMaker's hosted training functionality. Hosted training is preferred to for doing actual training, especially large-scale, distributed training.  Unlike Local Mode training, for hosted training the actual training itself occurs not on the notebook instance, but on a separate cluster of machines managed by SageMaker.  Before starting hosted training, the data must be uploaded to S3. We'll do that now, and confirm the upload was successful.

In [15]:
s3_prefix = 'tf-eager-scriptmode-bostonhousing'

traindata_s3_prefix = '{}/data/train'.format(s3_prefix)
testdata_s3_prefix = '{}/data/test'.format(s3_prefix)

In [16]:
train_s3 = sagemaker.Session().upload_data(path='./data/train/', key_prefix=traindata_s3_prefix)
test_s3 = sagemaker.Session().upload_data(path='./data/test/', key_prefix=testdata_s3_prefix)

inputs = {'train':train_s3, 'test': test_s3}

print(inputs)

{'train': 's3://sagemaker-us-west-2-973822232355/tf-eager-scriptmode-bostonhousing/data/train', 'test': 's3://sagemaker-us-west-2-973822232355/tf-eager-scriptmode-bostonhousing/data/test'}


We're now ready to set up an Estimator object for hosted training. It is similar to the Local Mode Estimator, except the `train_instance_type` has been set to a ML instance type instead of `local` for Local Mode. Also, since we know our code is working now, we train for a larger number of epochs.

With these two changes, we simply call `fit` to start the actual hosted training.

In [13]:
train_instance_type = 'ml.c5.xlarge'
hyperparameters = {'epochs': 30, 'batch_size': 128, 'learning_rate': 0.01}

estimator = TensorFlow(
                       git_config=git_config,
                       source_dir='tf-eager-script-mode/train_model',
                       entry_point='train.py',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-scriptmode-bostonhousing',
                       framework_version='1.13',
                       py_version='py3',
                       script_mode=True)

In [ ]:
estimator.fit(inputs)

2019-10-11 01:45:02 Starting - Starting the training job...

As with the Local Mode training, hosted training produces a model checkpoint saved in S3 that we can retrieve and load. We can then make predictions and compare them with the test set.  This also demonstrates the modularity of SageMaker: having trained the model in SageMaker, you can now take the model out of SageMaker and run it anywhere else.  Alternatively, you can deploy the model using SageMaker's hosted endpoints functionality.

Now, instead of using a Local Mode endpoint, we'll go through the steps of downloading the model from Amazon S3 and loading a checkpoint.  

In [17]:
!aws s3 cp {estimator.model_data} ./model/model.tar.gz

download: s3://sagemaker-us-west-2-973822232355/tf-scriptmode-bostonhousing-2019-10-11-01-45-00-661/output/model.tar.gz to model/model.tar.gz


In [18]:
!tar -xvzf ./model/model.tar.gz -C ./model

weights.ckpt.index
checkpoint
weights.ckpt.data-00000-of-00001
1570758458/
1570758458/assets/
1570758458/assets/saved_model.json
1570758458/saved_model.pb
1570758458/variables/
1570758458/variables/checkpoint
1570758458/variables/variables.data-00000-of-00001
1570758458/variables/variables.index


In [19]:
!mkdir -p train_model
!wget -q -P ./train_model https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/tf-eager-script-mode/train_model/model_def.py
from tensorflow.contrib.eager.python import tfe
from train_model import model_def

tf.keras.backend.clear_session()
device = '/cpu:0' 

with tf.device(device):    
    model = model_def.get_model()
    saver = tfe.Saver(model.variables)
    saver.restore('model/weights.ckpt')

In [20]:
with tf.device(device):   
    predictions = model.predict(x_test)
    
print('predictions: \t{}'.format(predictions[:10].flatten().round(decimals=1)))
print('target values: \t{}'.format(y_test[:10].round(decimals=1)))

predictions: 	[12.2 18.9 20.  29.7 21.9 20.9 26.1 21.2 19.3 29. ]
target values: 	[ 7.2 18.8 19.  27.  22.2 24.5 31.2 22.9 20.5 23.2]


##  SageMaker hosted endpoint

After multiple sanity checks, we're confident that our model is performing as expected. If we wish to deploy the model to production, a convenient option is to use a SageMaker hosted endpoint. The endpoint will retrieve the TensorFlow SavedModel created during training and deploy it within a TensorFlow Serving container. This all can be accomplished with one line of code, an invocation of the Estimator's deploy method.

In [21]:
predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m5.xlarge')

--------------------------------------------------------------------------!

As one last sanity check, we can compare the predictions generated by the endpoint with those generated locally by the model checkpoint we retrieved from hosted training in SageMaker. 

In [22]:
results = predictor.predict(x_test[:10])['predictions'] 
flat_list = [float('%.1f'%(item)) for sublist in results for item in sublist]
print('predictions: \t{}'.format(np.array(flat_list)))
print('target values: \t{}'.format(y_test[:10].round(decimals=1)))

predictions: 	[12.2 18.9 20.  29.7 21.9 20.9 26.1 21.2 19.3 29. ]
target values: 	[ 7.2 18.8 19.  27.  22.2 24.5 31.2 22.9 20.5 23.2]


Before proceeding with the rest of this notebook, you can delete the prediction endpoint to release the instance(s) associated with it.

In [23]:
sagemaker.Session().delete_endpoint(predictor.endpoint)

## Automatic Model Tuning

Selecting the right hyperparameter values to train your model can be difficult. The right answer is dependent on your data; some algorithms have many different hyperparameters that can be tweaked; some are very sensitive to the hyperparameter values selected; and most have a non-linear relationship between model fit and hyperparameter values.  SageMaker Automatic Model Tuning helps automate the hyperparameter tuning process:  it runs multiple training jobs with different hyperparameter combinations to find the set with the best model performance.

We begin by specifying the hyperparameters we wish to tune, and the range of values over which to tune each one.  We also must specify an objective metric to be optimized:  in this use case, we'd like to minimize the validation loss.

In [24]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from time import gmtime, strftime 

hyperparameter_ranges = {
        'learning_rate': ContinuousParameter(0.001, 0.2, scaling_type="Logarithmic"),
        'epochs': IntegerParameter(10, 50),
        'batch_size': IntegerParameter(64, 256),
    }

metric_definitions = [{'Name': 'loss',
                       'Regex': ' loss: ([0-9\\.]+)'},
                     {'Name': 'val_loss',
                       'Regex': ' val_loss: ([0-9\\.]+)'}]

objective_metric_name = 'val_loss'
objective_type = 'Minimize'

Next we specify a HyperparameterTuner object that takes the above definitions as parameters.  Each tuning job must be given a budget - a maximum number of training jobs - and the tuning job will complete once that many training jobs have been executed.  

We also can specify how much parallelism to employ, in this case five jobs, meaning that the tuning job will complete after three series of five jobs in parallel have completed.  For the default Bayesian Optimization tuning strategy used here, the search is informed by the results of previous groups of training jobs, so we don't run all of the jobs in parallel, but rather divide the jobs into groups of parallel jobs.  In other words, more parallel jobs will finish tuning sooner, but may sacrifice accuracy. 

Now we can launch a hyperparameter tuning job by calling the `fit` method of the HyperparameterTuner object. We will wait until the tuning finished, which may take around 10 minutes.

In [25]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=15,
                            max_parallel_jobs=5,
                            objective_type=objective_type)

tuning_job_name = "tf-bostonhousing-{}".format(strftime("%d-%H-%M-%S", gmtime()))
tuner.fit(inputs, job_name=tuning_job_name)
tuner.wait()

...............................................................................................................!


After the tuning job is finished, we can use the `HyperparameterTuningJobAnalytics` method to list the top 5 tuning jobs with the best performance. Although the results typically vary from tuning job to tuning job, the best validation loss from the tuning job (under the FinalObjectiveValue column) likely will be lower than the validation loss from the hosted training job above.  For an example of a more in-depth analysis of a tuning job, see HPO_Analyze_TuningJob_Results.ipynb notebook.

In [26]:
tuner_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
tuner_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=True).head(5)

,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,batch_size,epochs,learning_rate
2,15.596400,54.0,2019-10-11 02:42:21+00:00,tf-bostonhousing-11-02-33-42-013-e69a4a94,Completed,2019-10-11 02:41:27+00:00,112.0,49.0,0.022322
5,17.953899,46.0,2019-10-11 02:39:29+00:00,tf-bostonhousing-11-02-33-42-010-70fc1d6a,Completed,2019-10-11 02:38:43+00:00,117.0,48.0,0.039858
3,18.176701,55.0,2019-10-11 02:42:16+00:00,tf-bostonhousing-11-02-33-42-012-ac02d457,Completed,2019-10-11 02:41:21+00:00,119.0,48.0,0.037801
4,20.471901,46.0,2019-10-11 02:42:06+00:00,tf-bostonhousing-11-02-33-42-011-68598f77,Completed,2019-10-11 02:41:20+00:00,80.0,26.0,0.021741
11,21.852301,51.0,2019-10-11 02:36:35+00:00,tf-bostonhousing-11-02-33-42-004-052690ba,Completed,2019-10-11 02:35:44+00:00,64.0,11.0,0.024748


The total training time and training jobs status can be checked with the following script. Because automatic early stopping is by default off, all the training jobs should be completed normally.

In [27]:
total_time = tuner_metrics.dataframe()['TrainingElapsedTimeSeconds'].sum() / 3600
print("The total training time is {:.2f} hours".format(total_time))
tuner_metrics.dataframe()['TrainingJobStatus'].value_counts()

The total training time is 0.21 hours


Completed    15
Name: TrainingJobStatus, dtype: int64

Assuming the best model from the tuning job is better than the model produced by the hosted training job above, we could now easily deploy that model.  By calling the `deploy` method of the HyperparameterTuner object we instantiated above, we can directly deploy the best model from the tuning job to a SageMaker hosted endpoint:

`tuning_predictor = tuner.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')`

Since we already looked at how to use a SageMaker hosted endpoint above, we won't repeat that here.  We've covered a lot of content in this notebook:  local and hosted training with Script Mode, local and hosted inference in SageMaker, and Automatic Model Tuning.  These are likely to be central elements for most deep learning workflows in SageMaker.  